# Part 3: Single-View Geometry

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [ ]:
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image

# Provided functions

In [19]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    plt.show()
    print('Set at least %d lines to compute vanishing point' % min_lines)
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least %d lines, you have %d now' % (min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [20]:
def plot_lines_and_vp(im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10

    plt.figure()
    plt.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    plt.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    plt.show()

In [21]:
def get_top_and_bottom_coordinates(im, obj):
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
            where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is the homogeneous
            coordinate of the bottom
    """
    plt.figure()
    plt.imshow(im)

    print('Click on the top coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # Uncomment this line to enable a vertical line to help align the two coordinates
    # plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print('Click on the bottom coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    plt.plot([x1, x2], [y1, y2], 'b')

    return np.array([[x1, x2], [y1, y2], [1, 1]])

# Your implementation

In [22]:
def get_vanishing_point(lines):
    """
    Solves for the vanishing point using the user-input lines.
    """
    s = lines.dot(lines.T)
    w, v = np.linalg.eig(s)
    min_index = np.argmin(w)
    point = v[:, min_index]
    point = point / point[-1]
    return point

In [23]:
def get_horizon_line(points):
    """
    Calculates the ground horizon line.
    """
    p1 = points[:, 0]
    p2 = points[:, 1]
    line = np.cross(p1, p2)
    norm = np.sqrt(line[0]**2 + line[1]**2)
    return line / norm

In [24]:
def plot_horizon_line(image, line):
    """
    Plots the horizon line.
    """
    r = image.shape[1]
    x = np.arange(r)
    y = (-line[2]-line[0] * x) / line[1]
    plt.figure()
    plt.imshow(image)
    plt.plot(x, y)
    plt.show()

In [25]:
from sympy import solve, symbols, Matrix
def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    p1 = vpts[:, 0][:, np.newaxis]
    p2 = vpts[:, 1][:, np.newaxis]
    p3 = vpts[:, 2][:, np.newaxis]
    f, u, v = symbols("f, u, v")
    inverse_K_T = Matrix([[1/f, 0, 0], [0, 1/f, 0], [-u/f, -v/f, 1]])
    inverse_K = Matrix([[1/f, 0, -u/f], [0, 1/f, -v/f], [0, 0, 1]])

    e1 = p1.T * inverse_K_T * inverse_K * p2
    e2 = p1.T * inverse_K_T * inverse_K * p3
    e3 = p2.T * inverse_K_T * inverse_K * p3
    f, u, v = solve([e1[0], e2[0], e3[0]], (f, u, v))[0]
    f = abs(f)

    return f, u, v


In [26]:
def get_rotation_matrix(vpts, K):
    """
    Computes the rotation matrix using the camera parameters.
    """
    Y_dir = vpts[:, 2][:, np.newaxis]
    X_dir = vpts[:, 1][:, np.newaxis]
    Z_dir = vpts[:, 0][:, np.newaxis]

    inverse_K = np.array(K.inv()).astype(np.float)

    r1 = inverse_K.dot(X_dir)
    r2 = inverse_K.dot(Y_dir)
    r3 = inverse_K.dot(Z_dir)

    r1 = r1 / np.linalg.norm(r1)
    r2 = r2 / np.linalg.norm(r2)
    r3 = r3 / np.linalg.norm(r3)

    R = np.concatenate((r1, r2, r3), axis=1)
    return R

In [27]:
# def estimate_height(vpts, reference_height, reference_coord, object_coord):
#     """
#     Estimates height for a specific object using the recorded coordinates. You might need to plot additional images here for
#     your report.
#     """
#     #
#     vx = vpts[:, 0]
#     vy = vpts[:, 1]
#     vz = vpts[:, 2]
#     r = object_coord[:, 0]
#     t_0 = reference_coord[:, 0]
#     b = object_coord[:, 1]
#     b_0 = reference_coord[:, 1]
#     v = np.cross(np.cross(b, b_0), np.cross(vx, vy))
#     v = v / v[-1]
#     t = np.cross(np.cross(v, t_0), np.cross(r, b))
#     t = t / t[-1]
#     height = reference_height \
#     / ((np.linalg.norm(t - b) * np.linalg.norm(vz - r)) / (np.linalg.norm(r - b) * np.linalg.norm(vz - t)))
#     return height
def estimate_height(vpts, reference_height, reference_coord, obj_coord, horizon_line):
    """
    Estimates height for a specific object using the recorded coordinates. You might need to plot additional images here for
    your report.
    """
    vpz = vpts[:, 2]  # Vertical vp.
    t0 = reference_coord[:, 0]
    b0 = reference_coord[:, 1]

    # Obj coords.
    r = obj_coord[:, 0]
    b = obj_coord[:, 1]

    line_b0_b = np.cross(b0, b)
    v = np.cross(line_b0_b, horizon_line)
    v = v / v[-1]

    line_v_t0 = np.cross(v, t0)
    line_r_b = np.cross(r, b)
    t = np.cross(line_v_t0, line_r_b)
    t = t / t[-1]

    height = reference_height*(np.linalg.norm(r-b) * np.linalg.norm(vpz-t) /
              np.linalg.norm(t-b) / np.linalg.norm(vpz-r))

    return height

# Main function

In [29]:
im = np.asarray(Image.open('CSL.jpeg'))

# Part 1
# Get vanishing points for each of the directions
num_vpts = 3
vpts = np.zeros((3, num_vpts))
for i in range(num_vpts):
    print('Getting vanishing point %d' % i)
    # Get at least three lines from user input
    n, lines, centers = get_input_lines(im)
    # <YOUR IMPLEMENTATION> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(lines)
    print("coordinates: ", vpts[:, i])
    # Plot the lines and the vanishing point
    plot_lines_and_vp(im, lines, vpts[:, i])

# <YOUR IMPLEMENTATION> Get the ground horizon line
horizon_line = get_horizon_line(vpts)
# <YOUR IMPLEMENTATION> Plot the ground horizon line
plot_horizon_line(im, horizon_line)

Getting vanishing point 0
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Getting vanishing point 1
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Getting vanishing point 2
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the mi

In [ ]:

# Part 2
# <YOUR IMPLEMENTATION> Solve for the camera parameters (f, u, v)
f, u, v = get_camera_parameters(vpts)
print("f = {}, u = {}, v = {}".format(f, u, v))

# Part 3
# <YOUR IMPLEMENTATION> Solve for the rotation matrix
K = Matrix(((f, 0, u), (0, f, v), (0, 0, 1)))
R = get_rotation_matrix(vpts, K)
print(R)

# Part 4
# Record image coordinates for each object and store in map
objects = ('person', 'CSL building', 'the spike statue', 'the lamp posts')
coords = dict()
for obj in objects:
    coords[obj] = get_top_and_bottom_coordinates(im, obj)

# <YOUR IMPLEMENTATION> Estimate heights
reference_height = 66.0
for obj in objects[1:]:
    print('Estimating height of %s' % obj)
    height = estimate_height(vpts, reference_height, coords['person'], coords[obj], horizon_line)
    print("Height of {} is {}".format(obj, height))

reference_height = 72.0
for obj in objects[1:]:
    print('Estimating height of %s' % obj)
    height = estimate_height(vpts, reference_height, coords['person'], coords[obj], horizon_line)
    print("Height of {} is {}".format(obj, height))